# Webscraping Non-Text Data 

### Webscraping Supreme Court Justice Information from Constitution Annotated

In [1]:
import requests

url = 'https://constitution.congress.gov/resources/supreme-court-justices/'
response  = requests.get(url)
response.status_code

200

In [2]:
from bs4 import BeautifulSoup
import time, os
import pandas as pd 

In [3]:
page = response.text
soup = BeautifulSoup(page, 'html.parser')

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="A table listing all Justices who have served or are currently serving on the Supreme Court." name="description"/>
  <meta content="A table listing all Justices who have served or are currently serving on the Supreme Court." property="og:description"/>
  <meta content="https://constitution.congress.gov/static/images/og.jpg" property="og:image"/>
  <meta content="website" property="og:type"/>
  <meta content="Table of Supreme Court Justices | Resources | Constitution Annotated | Congress.gov | Library of Congress" property="og:title"/>
  <meta content="https://constitution.congress.gov/resources/supreme-court-justices/" property="og:url"/>
  <link href="/static/images/apple-touch-icon-precomposed.png" rel="image_src"/>
  <link href="https://www.congress.gov/favicon.ico" rel="shortcut icon">
   <link href="/static/css

In [5]:
def table_data(soup, class_):

    def get_data(tr, tag='td'):
        text_list = [td.get_text(strip=True) for td in tr.find_all(tag)[1:5]]
        return text_list

    rows = []

    table = soup.find('table', class_=class_)
    trs = table.find_all('tr')
    for tr in trs: 
        rows.append(get_data(tr, 'td'))
    return rows

In [6]:
table_contents = table_data(soup, 'table table-sticky-thead table-bordered')

justice_cols = ['Justice Name', 'Term Start Date', 'Term End Date', 'Appointing President']
    
justice_df = pd.DataFrame(table_contents[1:], columns=justice_cols)
    

In [7]:
import re

names_to_replace = {'Adams, John Quincy':'John Quincy Adams',
 'Van Buren, Martin':'Martin Van Buren',
 'Polk, James K.':'James K. Polk',
 'Grant, Ulysses S.':'Ulysses S. Grant',
 'Hayes, Rutherford B.':'Rutherford B. Hayes',
 'Garfield, James A.':'James A. Garfield',
 'Arthur, Chester A.':'Chester A. Arthur',
 'Taft, William Howard':'William Howard Taft',
 'Harding, Warren G.':'Warren G. Harding',
 'Truman, Harry S.':'Harry S. Truman',
 'Eisenhower, Dwight D.':'Dwight D. Eisenhower',
 'Kennedy, John F.':'John F. Kennedy',
 'Johnson, Lyndon B.':'Lyndon B. Johnson',
 'Nixon, Richard M.':'Richard M. Nixon',
 'Bush, George W.':'George W. Bush'}

justice_df['Appointing President'].replace(to_replace=names_to_replace, inplace=True)

def fix_names(name_string):
    fixed = ['John Quincy Adams',
             'Martin Van Buren',
             'William Henry Harrison',
             'James K. Polk',
             'Ulysses S. Grant',
             'Rutherford B. Hayes',
             'James A. Garfield',
             'Chester A. Arthur',
             'William Howard Taft',
             'Warren G. Harding',
             'Harry S. Truman',
             'Dwight D. Eisenhower',
             'John F. Kennedy',
             'Lyndon B. Johnson',
             'Richard M. Nixon',
             'George W. Bush']
    if name_string in fixed:
        pass
    else:
        name = re.sub(r'[^\w\s]','', name_string).split()
        first_name, last_name = name[1], name[0]
        return first_name + ' ' + last_name
    
justice_df['Appointing President'] = justice_df['Appointing President'].apply(fix_names)

### Converting Variables 'Term Start Date' and 'Term End Date' to datetime data type

In [8]:
from datetime import datetime
import string

In [9]:
justice_df[[justice_df['Term Start Date']=='October 10,1910']] = 'October 10, 1910'

In [10]:
def string_to_date(s):
    if s == '--':
        return s
    else:
        s = re.sub(r'[^\w\s]','', s)
        s_list = s.split(' ')
        if len(s_list[1]) == 1:
            s_list[1] = '0' + s_list[1]
        s = ' '.join(s_list)
    
        datetime_object = datetime.strptime(s, '%B %d %Y')
    
        return datetime_object

justice_df['Term Start Date'] = justice_df['Term Start Date'].apply(string_to_date)
justice_df['Term End Date'] = justice_df['Term End Date'].apply(string_to_date)

In [11]:
justice_df.head()

,Justice Name,Term Start Date,Term End Date,Appointing President
0,"Coney Barrett, Amy(Associate Justice)",2020-10-26,--,Donald Trump
1,"Kavanaugh, Brett M.(Associate Justice)",2017-10-10,--,Donald Trump
2,"Gorsuch, Neil M.(Associate Justice)",2017-04-10,--,Donald Trump
3,"Kagan, Elena(Associate Justice)",2010-08-07,--,Barack Obama
4,"Sotomayor, Sonia(Associate Justice)",2009-08-08,--,Barack Obama


In [12]:
justice_df['Appointing President'].unique()

array(['Donald Trump', 'Barack Obama', None, 'William Clinton',
       'George Bush', 'Ronald Reagan', 'Gerald Ford',
       'Franklin Roosevelt', 'Herbert Hoover', 'Calvin Coolidge',
       'Woodrow Wilson', 'October 10, 1910', 'Theodore Roosevelt',
       'William McKinley', 'Grover Cleveland', 'Benjamin Harrison',
       'Abraham Lincoln', 'James Buchanan', 'Franklin Pierce',
       'Millard Fillmore', 'John Tyler', 'Andrew Jackson',
       'Thomas Jefferson', 'James Monroe', 'James Madison',
       'George Washington'], dtype=object)

In [18]:
justice_df.to_csv('justices.csv', index=False)

### Webscraping Parties of Presidents from The Guardian 

In [13]:
url_parties = 'https://www.theguardian.com/news/datablog/2012/oct/15/us-presidents-listed'
response_parties  = requests.get(url_parties)
page_parties = response_parties.text
soup_parties = BeautifulSoup(page_parties, 'html.parser')

In [14]:
pres_party_table = table_data(soup_parties, 'in-article sortable')

party_cols = ['President', 'Party']
    
party_df = pd.DataFrame(pres_party_table[1:], columns=party_cols)

party_df.drop_duplicates(inplace=True, ignore_index=True)

In [15]:
problem_names = []
for index, value in party_df['President'].iteritems():
    name = value.split()
    if len(name) > 2:
        name = ' '.join(name)
        problem_names.append(name)
problem_names

['John Quincy Adams',
 'Martin Van Buren',
 'William Henry Harrison',
 'James K. Polk',
 'Ulysses S. Grant',
 'Rutherford B. Hayes',
 'James A. Garfield',
 'Chester A. Arthur',
 'William Howard Taft',
 'Warren G. Harding',
 'Harry S. Truman',
 'Dwight D. Eisenhower',
 'John F. Kennedy',
 'Lyndon B. Johnson',
 'Richard M. Nixon',
 'George W. Bush']

In [16]:
party_df.loc[43] = ['Donald Trump', 'Republican']
party_df.loc[44] = ['Joe Biden','Democrat']

In [19]:
party_df.to_csv('pres_parties.csv', index=False)

# Converting Data to SQL database

In [17]:
import sqlalchemy as sql
from sqlalchemy_utils import create_database, database_exists

In [18]:
engine = sql.create_engine('sqlite:///supreme_court.db')
if not database_exists(engine.url):
    create_database(engine.url)


print(database_exists(engine.url))

True


In [19]:
justice_df.to_sql('justices', con=engine, index=False, if_exists='replace')
party_df.to_sql('pres_party', con=engine, index=False, if_exists='replace')

In [20]:
metadata = sql.MetaData()
metadata.reflect(bind=engine)
for t in metadata.sorted_tables:
    print(t)
connection = engine.connect()

justices
pres_party


In [21]:
justices = sql.Table('justices', metadata, autoload=True, autoload_with=engine)
query = sql.select([justices])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet[:3]

[('Coney Barrett, Amy(Associate Justice)', datetime.datetime(2020, 10, 26, 0, 0), '--', 'Donald Trump'),
 ('Kavanaugh, Brett M.(Associate Justice)', datetime.datetime(2017, 10, 10, 0, 0), '--', 'Donald Trump'),
 ('Gorsuch, Neil M.(Associate Justice)', datetime.datetime(2017, 4, 10, 0, 0), '--', 'Donald Trump')]